# TP1

**Parte 1**


**1.1 - Pasaje a coordenadas cromáticas**

In [1]:
%matplotlib qt

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt # pip install PyQt5

In [41]:
def chromaticCoordinates(imgRGB):#castear todo a float las 6 variables
    im_r, im_g, im_b = cv.split(imgRGB)
    
    CC_r = np.zeros((im_r.shape[0], im_r.shape[1]))
    CC_g = np.zeros((im_r.shape[0], im_r.shape[1]))
    CC_b = np.zeros((im_r.shape[0], im_r.shape[1]))

    im_r = im_r.astype(float)
    im_g = im_g.astype(float)
    im_b = im_b.astype(float)
    
    CC_r = CC_r.astype(float)
    CC_g = CC_g.astype(float)
    CC_b = CC_b.astype(float)

    for column in range(0, im_r.shape[0]):
        for row in range(0, im_r.shape[1]):
            R_pixel = im_r[column][row]
            G_pixel = im_g[column][row]
            B_pixel = im_b[column][row]
            currentPixelSum = float(R_pixel+G_pixel+B_pixel)
            #print("R: ",R_pixel,",G: ",G_pixel,",B: ",B_pixel,"=",currentPixelSum)
            if currentPixelSum == 0:
                CC_r[column][row] = 0
                CC_g[column][row] = 0
                CC_b[column][row] = 0
                #print("Valor de currentPixelSum es 0 en [",column,"][",row,"]")
            else:
                CC_r[column][row] = float(R_pixel/currentPixelSum)
                CC_g[column][row] = float(G_pixel/currentPixelSum)
                CC_b[column][row] = float(B_pixel/currentPixelSum)
    imgCC = cv.merge((CC_r, CC_g, CC_b))
    plt.imshow(imgCC)
    plt.show()

In [42]:
img = cv.imread("./coord_cromaticas/CoordCrom_2.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
chromaticCoordinates(imgRGB)

**1.2 - White Patch**

In [62]:
def normalize(channel):
    min_value = min(channel.flatten())
    max_value = max(channel.flatten())
    for column in range(0, channel.shape[0]):
        for row in range(0, channel.shape[1]):
            valor = channel[column][row]
            channel[column][row] = (valor - min_value) / \
                (max_value - min_value) * 255
    return np.uint8(channel)

In [63]:
def setWhitePatch(imgRGB):
    im_r, im_g, im_b = cv.split(imgRGB)
    R_max_value = max(im_r.flatten())
    # print("Max value in R ",R_max_value);
    G_max_value = max(im_g.flatten())
    # print("Max value in G ",G_max_value);
    B_max_value = max(im_b.flatten())
    # print("Max value in B ",B_max_value);
    R_channel = normalize(np.uint8((255/R_max_value)*im_r))
    G_channel = normalize(np.uint8((255/G_max_value)*im_g))
    B_channel = normalize(np.uint8((255/B_max_value)*im_b))
    imgWhitePatch = cv.merge((R_channel, G_channel, B_channel))
    plt.imshow(imgWhitePatch)
    plt.show()

In [64]:
img = cv.imread("./white_patch/test_red.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [58]:
img = cv.imread("./white_patch/test_green.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [59]:
img = cv.imread("./white_patch/test_blue.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [56]:
img = cv.imread("./white_patch/wp_red.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [57]:
img = cv.imread("./white_patch/wp_red2.jpg")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [60]:
img = cv.imread("./white_patch/wp_green.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [51]:
img = cv.imread("./white_patch/wp_green2.jpg")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

In [61]:
img = cv.imread("./white_patch/wp_blue.jpg")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
setWhitePatch(imgRGB)

Podemos concluir que para las primeras imagenes test_blue, test_green y test_red
se pudo eliminar por completo los colores y aplicamos con exito el white patch.
Mientras que para las siguientes imagenes con prefijo wp no tuvimos tanto exito.
en el caso de las imagenes mas pequeñas se nota mas el trabajo del filtro pero
en las imagenes (wp_blue.jpg, wp_green2.jpg, wp_red2.jpg) no se puede denotar
mucha diferencia con las imagenes originales.

Parte 2

**2.1 - Leer con OpenCV en escala de grises y visualizar imagenes**

In [28]:
img1 = cv.imread('img1_tp.png', cv.IMREAD_GRAYSCALE)
img2 = cv.imread('img2_tp.png', cv.IMREAD_GRAYSCALE)
cv.imshow("img1",img1)
cv.imshow("img2",img2)


**2.2 - Histogramas**

In [31]:
def pltimg(img,title):
    # Nueva figura
    fig = plt.figure()

    # Imagen original
    ax1=plt.subplot(221)
    ax1.imshow(img, cmap='gray', vmin=0, vmax=255)
    ax1.set_title(title)

    hist1,bins1 = np.histogram(img.ravel(),256,[0,256])
    ax3=plt.subplot(223)
    ax3.plot(hist1)
    fig.show()

In [32]:
pltimg(img1,'img1')
pltimg(img2,'img2')

Se puede denotar que los histogramas muy similares para no decir iguales.
Creo que no seria buena idea tomar los histogramas com feaures, porque
puede pasar como en este caso dos imagenes visiblemente diferentes cuentan con
el histograma muy semejante, no teniendo asi caracteristicas identificatorias
para cada una de las imagenes.

**2.3 - Segmentacion**

In [65]:
def rgbHistograms(imgRGB):
    im_r, im_g, im_b = cv.split(imgRGB)
    pltimg(im_r,'R Channel')
    pltimg(im_g,'G Channel')
    pltimg(im_b,'B Channel')

In [66]:
img = cv.imread("./segmentacion.png")
imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
rgbHistograms(imgRGB)

In [67]:
def imgSegmentation(img_muestra,imgRGB,colorType,l_Inferior,l_Superior):
    img_HSV = cv.cvtColor(imgRGB, colorType)
    color_mean = cv.mean(img_muestra)
    print(color_mean)

    color_l = (color_mean[0]-l_Inferior[0],l_Inferior[1],l_Inferior[2])
    color_u = (color_mean[0]+l_Superior[0],l_Superior[1],l_Superior[2])

    mask = cv.inRange(img_HSV, color_l,  color_u)
    img_segmentada = cv.bitwise_and(imgRGB, imgRGB, mask=mask)

    plt.figure(6)
    plt.imshow(img_segmentada)
    plt.show()    

In [68]:
#Tomamos la muestra del cielo
img_muestra1 = imgRGB[50:200,500:900,:]
plt.figure(2)
plt.imshow(img_muestra1)
plt.show()

In [69]:
#Tomamos la muestra de la costa
img_muestra2 = imgRGB[450:600,150:500,:]
plt.figure(2)
plt.imshow(img_muestra2)
plt.show()

In [70]:
#Tomamos la muestra del mar
img_muestra3 = imgRGB[305:380,450:900,:]
plt.figure(2)
plt.imshow(img_muestra3)
plt.show()

In [71]:
# Definimos el limite superior y el limite inferior para la segmentacion de 
# la muestra 1
l_Inferior=[20,20,150]
l_Superior=[25,255,255]
imgSegmentation(img_muestra1,imgRGB,cv.COLOR_RGB2HSV,l_Inferior,l_Superior)

(82.69023333333334, 170.42413333333334, 231.817, 0.0)


In [73]:
# Definimos el limite superior y el limite inferior para la segmentacion de 
# la muestra 2
l_Inferior=[0,0,0]
l_Superior=[250,255,180]
imgSegmentation(img_muestra2,imgRGB,cv.COLOR_RGB2HSV,l_Inferior,l_Superior)

(94.99607619047619, 74.2892, 45.42337142857143, 0.0)


In [72]:
# Definimos el limite superior y el limite inferior para la segmentacion de 
# la muestra 3
l_Inferior=[10,0,0]
l_Superior=[30,255,255]
imgSegmentation(img_muestra3,imgRGB,cv.COLOR_BGR2HSV,l_Inferior,l_Superior)

(74.71146666666667, 122.25795555555555, 135.83155555555555, 0.0)


Pudimos realizar el trabajo de segmentacion de las 3 muestras, teniendo mas
dificultad entre el mar y el cielo ya que sus colores son similares, pero
definiendo los limites correctos es posible realizar una buena segmentacion.